<a href="https://colab.research.google.com/github/CristianoSouza/tcc/blob/master/teste_dos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### *João Vitor Cardoso <2020>*

# **IDS - Deep Learning (DOS)**

  Usando a base [CSE-CIC-IDS2018](https://www.unb.ca/cic/datasets/ids-2018.html) para detecção de intrusão com redes neurais, para detectar ataques de negação de serviço (DOS)
  

## Baixando Base da Dados

## Importando Bibliotecas

Importando bibliotecas

In [0]:
#justtesting
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


### Verificando GPU

In [0]:
tf.test.gpu_device_name()

''

In [0]:
!nvidia-smi

Wed Apr 22 20:46:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Explorando os Dados

### Carregando dados

In [0]:
# As bases contem varios tipos de ataque, não só DoS
train_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Train.csv'
test_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Test.csv'
#define o nome das colunas de dados
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

#le o CSV das URLs de acordo com as colunas
df = pd.read_csv(train_url,header=None, names = col_names)
df_test = pd.read_csv(test_url, header=None, names = col_names)

In [0]:
dos = dos1.append(dos2, ignore_index=True)

### Tamanho dos dados

In [0]:
df.shape

(125973, 42)

### Visualisando dados

In [0]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


### Analisando tipos de ataques (porcentagem)



In [0]:
df.label.value_counts()/sum(df.label.value_counts())*100

normal             53.458281
neptune            32.716534
satan               2.883951
ipsweep             2.856961
portsweep           2.326689
smurf               2.100450
nmap                1.185175
back                0.758893
teardrop            0.708088
warezclient         0.706501
pod                 0.159558
guess_passwd        0.042073
buffer_overflow     0.023815
warezmaster         0.015876
land                0.014289
imap                0.008732
rootkit             0.007938
loadmodule          0.007144
ftp_write           0.006351
multihop            0.005557
phf                 0.003175
perl                0.002381
spy                 0.001588
Name: label, dtype: float64

## Pré-processamento dos dados





Forma dos dados antes do  pre 
processamento

In [0]:
print("mostrar forma antes da categorização:")
df.head()

In [0]:
from sklearn.preprocessing import LabelEncoder
import keras.preprocessing.text


columns_category = ['protocol_type','service','flag']

le = LabelEncoder()
for col in columns_category:
  le.fit(df[col])
  #self.classes[col]= le.classes_
  df[col] = le.fit_transform(df[col])
  df_test[col] = le.fit_transform(df_test[col]) 

print("mostrar forma depois da categorização:")
df.head()

mostrar forma depois da categorização:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


## Criando Modelo

Visualizando tamanho da entrada

In [0]:
df.shape[1]

42

Criando função para construção do modelo

In [0]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(15, activation='relu', input_shape=[42]),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  return model

Criando modelo

In [0]:
model = build_model()

Visualizando descrição do modelo

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                645       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
Total params: 860
Trainable params: 860
Non-trainable params: 0
_________________________________________________________________


### Criando checkpoint para salvar treinamento

Criando pasta para salvar arquivos

In [0]:
!mkdir training

Definindo caminho dos arquivos

In [0]:
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

Definindo função para salvar os pesos do treinamento

In [0]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Treinamento

In [0]:
history = model.fit(df, df_, epochs=10, callbacks=[cp_callback])

ValueError: ignored

### Validação

In [0]:
test_loss, test_acc = model.evaluate(df_test,  test_targ, verbose=2)

### Analise do treinamento

Recuperando valores de 'accuracy' e 'loss' durante o treinamento

In [0]:
acc = history.history['acc']
loss = history.history['loss']

Criando lista enumerando épocas

In [0]:
epochs = range(1, len(acc) + 1)

Criando gráfico para accuracy

In [0]:
plt.clf()

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.title('Training  accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

Criando gráfico para loss

In [0]:
plt.clf()

plt.plot(epochs, loss, '-bo', label='Training loss')
plt.title('Training  loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

## Restaurando rede neural apartir dos dados salvos

Verificando se os arquvios existem

In [0]:
!ls {checkpoint_dir}

Criando novo modelo e validando

In [0]:
model = build_model()

loss, acc = model.evaluate(test_feat,  test_targ, verbose=2)
print("Modelo sem treinamento, precisão: {:5.2f}%".format(100*acc))

Carregando pesos e validando

In [0]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_feat,  test_targ, verbose=2)
print("Modelo restaurado, precisão: {:5.2f}%".format(100*acc))

### Salvando dados no Drive

Conectando ao google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Compactando arquivos

In [0]:
!zip training.zip training/

Copiando para o google drive

In [0]:
!cp training.zip "/content/drive/My Drive/"